In [305]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, scale
import numpy as np
from scipy.stats import ttest_rel
import liwc

In [402]:
sentence_batch = 1
df = pd.read_csv('../data/annotation_study/MELD_annotation_trial_S{}_dev.csv'.format(sentence_batch))
df.dropna(axis=1, thresh=4, inplace=True)
df.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,LocationLatitude,...,21_QS1C5_2,21_QS1C5_3,21_QS1C5_4,21_QS1C5_5,21_QS1C5_6,21_QS1C5_7,21_QS1C5_8,CategoryBatch,PROLIFIC_PID,SentenceBatch
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Location Latitude,...,Sentence: [Field-1]\n \n\nFor each of the foll...,Sentence: [Field-1]\n \n\nFor each of the foll...,Sentence: [Field-1]\n \n\nFor each of the foll...,Sentence: [Field-1]\n \n\nFor each of the foll...,Sentence: [Field-1]\n \n\nFor each of the foll...,Sentence: [Field-1]\n \n\nFor each of the foll...,Sentence: [Field-1]\n \n\nFor each of the foll...,CategoryBatch,PROLIFIC_PID,SentenceBatch
1,"{""ImportId"":""startDate"",""timeZone"":""America/De...","{""ImportId"":""endDate"",""timeZone"":""America/Denv...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""locationLatitude""}",...,"{""ImportId"":""21_QID17_10""}","{""ImportId"":""21_QID17_16""}","{""ImportId"":""21_QID17_21""}","{""ImportId"":""21_QID17_23""}","{""ImportId"":""21_QID17_24""}","{""ImportId"":""21_QID17_25""}","{""ImportId"":""21_QID17_26""}","{""ImportId"":""CategoryBatch""}","{""ImportId"":""PROLIFIC_PID""}","{""ImportId"":""SentenceBatch""}"
2,2022-03-03 00:56:29,2022-03-03 01:00:59,IP Address,24.178.23.151,100,269,True,2022-03-03 01:00:59,R_6h5LhgnSXkExhuN,43.1204986572265625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,54e32a60fdf99b3eae9de85b,1
3,2022-03-03 00:55:05,2022-03-03 01:04:17,IP Address,135.23.65.140,100,552,True,2022-03-03 01:04:18,R_2e3TpesTfmkRlzU,43.768707275390625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,5fd315b7f2515a150aad2a10,1
4,2022-03-03 00:52:13,2022-03-03 01:04:18,IP Address,172.97.153.220,100,724,True,2022-03-03 01:04:18,R_1f8jV3L6je9hFu5,43.654693603515625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,60185baa7d10160c486bc4bc,1


In [403]:
df['SentenceBatch']

0                    SentenceBatch
1     {"ImportId":"SentenceBatch"}
2                                1
3                                1
4                                1
5                                1
6                                1
7                                1
8                                1
9                                1
10                               1
11                               1
12                               1
13                               1
14                               1
15                               1
16                               1
17                               1
18                               1
19                               1
20                               1
21                               1
22                               1
23                               1
24                               1
25                               1
26                               1
27                               1
28                  

In [404]:
parse, category_names = liwc.load_token_parser('../liwc_dict/LIWC2015_English_Flat.dic')
category_names = category_names[21:]

In [405]:
def wide_to_long(df, sentence_batch, category_batch, save=False):
    df_c1 = df[df['SentenceBatch'] == str(sentence_batch)]
    df_c1 = df_c1[df_c1['CategoryBatch'] == str(category_batch)].dropna(axis=1, how='all')
    attn_cols = [col for col in df_c1.columns if col.startswith('21_')]
    df_c1.drop(attn_cols, axis=1, inplace=True)
    rel_cols = [col for col in df_c1.columns if '_QS' in col]
    rel_cols_id = ['QProlificID'] + rel_cols
    df_c1 = df_c1[rel_cols_id].replace(
        'Not expressed', 0).replace(
        'Potentially expressed', 1).replace(
        'Most likely expressed', 2)
    df_c1_long = pd.melt(df_c1, id_vars='QProlificID', value_vars=rel_cols,
                         var_name='Object', value_name='Score')
    df_c1_long.columns = ['Rater', 'Object', 'Score']
    
    if save:
        df_c1_long.dropna(axis=0).to_csv('../data/annotation_study/dev_MELD_trial_S{}C{}_R6_long.csv'.format(sentence_batch,
                                                                                              category_batch), 
                          index=False)
    
    return df_c1_long

In [406]:
def process_str(col):
    parts = col.split('_')[:-1]
    parts[0] = parts[0].zfill(2)
    col = '_'.join(parts)
    return col

In [407]:
def name_categories(df_c1, categories):
    df_c1['Category'] = 'EMPTY'
    for i in range(1, len(categories)+1):
        df_c1.loc[df_c1['Object'].str.endswith('_{}'.format(i)), 'Category'] = categories[i-1]
    
    avg_df_c1 = df_c1.groupby(['Object', 'Category']).mean().reset_index()
    avg_df_c1['Object'] = avg_df_c1['Object'].apply(process_str)
    avg_df_c1_wide = avg_df_c1.pivot(index='Object', columns='Category', values='Score').reset_index()
    # avg_df_c1_wide.drop(20, axis=0, inplace=True)
    
    X_annotation = avg_df_c1_wide.drop('Object', axis=1)
    
    return X_annotation

In [408]:
def scale_df(df):
    df = scale(df, axis=1, with_mean=True, with_std=True, copy=True)
    scaler = StandardScaler()
    df = scaler.fit_transform(df)
    df = pd.DataFrame(df)
    
    return df

In [409]:
# for i in range(1, 6):
#     wide_to_long(df, i, save=True)

In [410]:
df_c1 = wide_to_long(df, sentence_batch, 1, True)
df_c2 = wide_to_long(df, sentence_batch, 2, True)
df_c3 = wide_to_long(df, sentence_batch, 3, True)
df_c4 = wide_to_long(df, sentence_batch, 4, True)
df_c5 = wide_to_long(df, sentence_batch, 5, True)

In [411]:
categories1 = ['netspeak', 'differ', 'cause', 'nonflu', 'discrep', 'drives', 'relig', 'swear', 'feel', 'home', 'family']
avg_df_c1_wide = name_categories(df_c1, categories1)

In [412]:
categories2 = ['leisure', 'sexual', 'see', 'bio', 'certain', 'money', 'percept', 'female', 'death', 'anger', 'cogproc']
avg_df_c2_wide = name_categories(df_c2, categories2)

In [413]:
categories3 = ['filler', 'sad', 'posemo', 'friend', 'relativ', 'ingest', 'body', 'work', 'time', 'social', 'informal']
avg_df_c3_wide = name_categories(df_c3, categories3)

In [414]:
categories4 = ['focusfuture', 'anx', 'affiliation', 'motion', 'power', 'reward', 'space', 'tentat', 'risk', 'focuspresent', 'affect']
avg_df_c4_wide = name_categories(df_c4, categories4)

In [415]:
categories5 = ['negemo', 'hear', 'male', 'health', 'insight', 'achiev', 'focuspast', 'assent']
avg_df_c5_wide = name_categories(df_c5, categories5)

In [416]:
X_annotation = pd.concat([avg_df_c1_wide, avg_df_c2_wide, avg_df_c3_wide, 
                         avg_df_c4_wide, avg_df_c5_wide], axis=1).sort_index(axis=1)

In [417]:
X_annotation = scale_df(X_annotation)

In [418]:
train_sent = pd.read_csv('../data/MELD/train_sent_emo.csv',
                        index_col=0).reset_index().drop('Sr No.', axis=1)
dev_sent = pd.read_csv('../data/MELD/dev.csv',
                        index_col=0).reset_index().drop('Sr No.', axis=1)
# sentence_df = pd.read_csv('../data/annotation_study/dev_sentence_sample_1.csv')
sentence_df = pd.read_excel('../data/annotation_study/dev_sentence_sample_{}.xlsx'.format(sentence_batch),
                           engine='openpyxl')

In [419]:
X_dev = pd.read_csv('../data/MELD/train_sentence_LIWC_centroid_cosine_all-mpnet-base-v2_refLIWCVocab.csv', index_col=0).drop(
    ['Sentiment', 'Emotion'], axis=1).sort_index(axis=1)
X_dev = X_dev[category_names].sort_index(axis=1)
X_dev = scale_df(X_dev)

X_devplus = pd.read_csv('../data/MELD/train_sentence_LIWC_centroid_cosine_all-mpnet-base-v2_refMELD.csv', index_col=0).drop(
    ['Sentiment', 'Emotion'], axis=1).sort_index(axis=1)
X_devplus = X_devplus[category_names].sort_index(axis=1)
X_devplus = scale_df(X_devplus)

X_liwc = pd.read_csv('../data/MELD/dev_liwc.csv', index_col=0).drop(
    ['sentiment', 'emotion'], axis=1).sort_index(axis=1)
X_liwc = scale_df(X_liwc)

X_liwc_soft = pd.read_csv('../data/MELD/dev_word2vec_liwc_counts.csv', index_col=0).drop(
    ['Sentiment', 'Emotion'], axis=1).sort_index(axis=1)
X_liwc_soft = scale_df(X_liwc_soft)

In [420]:
dev_overlaps = dev_sent[dev_sent['Utterance'].isin(
    sentence_df['Utterance'].values)]['Utterance'].drop_duplicates()

In [421]:
annot_overlaps = sentence_df[sentence_df['Utterance'].isin(dev_sent[dev_sent['Utterance'].isin(
    sentence_df['Utterance'].values)]['Utterance'])]['Utterance']

In [422]:
sentences = []
s_liwc_corr = []
s_liwc_plus_corr = []
liwc_corr = []
liwc_soft_corr = []

for i in range(len(dev_overlaps)):
    ref_idx = dev_overlaps.index[i]
    sentence = dev_overlaps.iloc[i]
    annotation_idx = annot_overlaps[annot_overlaps.isin([dev_overlaps.iloc[i]])].index[0]
    
    sentences.append(sentence)
    s_liwc_corr.append(round(np.corrcoef(X_annotation.iloc[annotation_idx].values, X_dev.iloc[ref_idx].values)[0, 1], 3))
    s_liwc_plus_corr.append(round(np.corrcoef(X_annotation.iloc[annotation_idx].values, X_devplus.iloc[ref_idx].values)[0, 1], 3))
    liwc_corr.append(round(np.corrcoef(X_annotation.iloc[annotation_idx].values, X_liwc.iloc[ref_idx].values)[0, 1], 3))
    liwc_soft_corr.append(round(np.corrcoef(X_annotation.iloc[annotation_idx].values, X_liwc_soft.iloc[ref_idx].values)[0, 1], 3))

In [423]:
corr_df = pd.DataFrame({'sentence': sentences, 
                        'sentecon_liwc_corr': s_liwc_corr,
                        'senteconplus_liwc_corr': s_liwc_plus_corr, 
                        'liwc_corr': liwc_corr,
                        'liwc_soft_corr': liwc_soft_corr})
corr_df

,sentence,sentecon_liwc_corr,senteconplus_liwc_corr,liwc_corr,liwc_soft_corr
0,What?,-0.054,0.251,0.112,0.218
1,Really?!,0.175,-0.011,0.211,-0.153
2,Hi Ross!,0.448,0.361,0.178,0.283
3,It’s really sweet and-and tender.,0.273,0.325,0.001,0.284
4,Tell her to wear her own earrings.,0.307,0.445,0.222,0.239
5,Yeah.,0.485,0.612,0.612,0.236
6,we talked through most of the night and we rea...,0.425,0.360,0.318,0.001
7,This is totally your fault!,0.663,0.672,0.453,0.358
8,My first time with Carol was...,0.456,0.487,0.166,0.234
9,"Yeah, right. Sure!",0.196,0.263,0.038,0.246


In [424]:
corr_df.to_csv('../data/annotation_study/MELD_trial_corrs_S{}_dev.csv'.format(sentence_batch), index=False)

In [425]:
np.mean(corr_df, axis=0)

sentecon_liwc_corr        0.28815
senteconplus_liwc_corr    0.30740
liwc_corr                 0.19455
liwc_soft_corr            0.23590
dtype: float64

In [160]:
# Paired two-sided t-test
ttest_rel(corr_df['sentence_liwc_corr'].values, corr_df['liwc_corr'].values)

KeyError: 'sentence_liwc_corr'

In [ ]:
# Paired one-sided t-test
ttest_rel(corr_df['sentence_liwc_corr'].values, corr_df['liwc_corr'].values,
         alternative='greater')